In [2]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [3]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

*First find the expression for the likelihood ratio:*


$
\begin{align}
\Lambda&=\frac{L(\lambda_0\mid\textbf{x})}{L(\hat{\lambda}_{ML}\mid\textbf{x})}\\
&=\frac{\prod_{i=1}^n f(x_i;\lambda=3)}{\prod_{i=1}^n f(x_i;\lambda=\frac{1}{\bar{X}})}\\
&=\frac{\prod_{i=1}^n\left(\lambda e^{-\lambda x_i}\right)_{\lambda=3}}{\prod_{i=1}^n\left(\lambda e^{-\lambda x_i}\right)_{\lambda=\frac{1}{\bar{X}}}}\\
&=\frac{\left(\lambda^n e^{\left(-\lambda\sum_{i=1}^n x_i\right)}\right)_{\lambda=3}}{\left(\lambda^n e^{\left(-\lambda\sum_{i=1}^n x_i\right)}\right)_{\lambda=\frac{1}{\bar{X}}}}\\
\end{align}
$

In [4]:
# Get values to plug in
xmean = np.mean(my_data)
xsum = np.sum(my_data)
ltest = 3
lmax = 1/xmean
n=20

# Calculate the likelihood ratio
L=(ltest**n * e**(-ltest*xsum) ) / (lmax**n * e**(-lmax*xsum) )
print("The Likelihood Ratio of null hypothesis is " + str(L))

# Calculate the p-value of -2log(L) under the chi-squared distribution
p = 1-stats.chi2.cdf(x=-2*log(L), df=1)
print("The p-value for the null hypothesis is " + str(p))

The Likelihood Ratio of null hypothesis is 0.0944569427967814
The p-value for the null hypothesis is 0.029827229194775096


*Since the likelihood ratio of the null hypothesis is very small, with a p-value less than 0.05, we have evidence to reject the null hypothesis.*

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [5]:
pcrit = 0.05 # this is the p-value below which we would reject H0
critValue = stats.chi2.ppf(q=1-pcrit,df=1)
critValue # any test statistic ABOVE this value will indicate a rejected null hypothesis

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [6]:
ltrue=5
ltest=3
rejections = make_array()

for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltrue, size=n) # get sample

    xmean = np.mean(sample) # get values to plug into L
    xsum = np.sum(sample)
    lmax = 1/xmean

    L=(ltest**n * e**(-ltest*xsum) ) / (lmax**n * e**(-lmax*xsum) )
    testStat = -2*log(L) # Get test statitic
    rejections = np.append(rejections, testStat>critValue) # record whether hypothesis gets rejected
    
np.mean(rejections) # fraction of tests that correctly reject the null hypothesis

0.6054

Repeat for a sample size of 50. What do you expect to happen to power? 

In [45]:
rejections = make_array()
n=50

for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltrue, size=n) # get sample

    xmean = np.mean(sample) # get values to plug into L
    xsum = np.sum(sample)
    lmax = 1/xmean

    L=(ltest**n * e**(-ltest*xsum) ) / (lmax**n * e**(-lmax*xsum) )
    testStat = -2*log(L) # Get test statitic
    rejections = np.append(rejections, testStat>critValue) # record whether hypothesis gets rejected
    
np.mean(rejections) # fraction of tests that correctly reject the null hypothesis

0.9954

*The power is much greater, indicating that an incorrect rejection of the null hypothesis is now very unlikely.*

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [46]:
# New Test Statistic: abs(3 - 1/mean of sample)
    # small values indicate sample is in accordance with H0
    # large values indicate sample is not in accordance with H0

# PART 1: empirical distribution of test stat under H0

testStats = make_array()
n=20
for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltest, size=n) # get sample using H0 lambda
    testStats = np.append(testStats, np.abs(ltest - 1/np.mean(sample))) # record test statistic
    
# PART 2: get p-value of observed test statistic
obsTest = np.abs(ltest - 1/np.mean(my_data))
p = np.sum(testStats>obsTest) / len(testStats)
print('The p-value of the observed data is ' + str(p))

The p-value of the observed data is 0.0156


How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

*The $p$-value of this test was somewhat less than the LRT $p$-value, leading to the same conclusion of rejecting the null hypothesis.*

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [47]:
# To get the value in testStats corresponding to the p-th percentile,
    # order testStats, and find the value in the p*len(testStats) position
critValue = np.sort(testStats)[int((1-pcrit)*len(testStats))]
critValue
np.sum(testStats>critValue) / len(testStats) # check by calculating p-value of critical value
# Our rejection region will be any test statistic GREATER then the critical value.

1.4730755714624273

0.0499

Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [48]:
rejections = make_array()

for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltrue, size=n) # get sample using TRUE lambda
    testStat = np.abs(ltest - 1/np.mean(sample))
    rejections = np.append(rejections, testStat>critValue) # record whether hypothesis gets rejected

np.mean(rejections) # fraction of tests that correctly reject the null hypothesis

0.7165

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [49]:
n=50
# Get new critical value
testStats = make_array()
for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltest, size=n) # get sample using H0 lambda
    testStats = np.append(testStats, np.abs(ltest - 1/np.mean(sample))) # record test statistic
critValue = np.sort(testStats)[int((1-pcrit)*len(testStats))]
critValue

# Retest power
rejections = make_array()

for _ in np.arange(10000):
    sample=stats.expon.rvs(scale=1/ltrue, size=n) # get sample using TRUE lambda
    testStat = np.abs(ltest - 1/np.mean(sample))
    rejections = np.append(rejections, testStat>critValue) # record whether hypothesis gets rejected

np.mean(rejections) # fraction of tests that correctly reject the null hypothesis

0.8640884631745305

0.9737